In [ ]:
import pandas as pd
import torch
import numpy as np
import pyntcloud
import pandas as pd 

In [3]:
model_path = "../results/south-building/ckpts/ckpt_6999_rank0.pt"
g_model = torch.load(model_path, map_location="cpu", weights_only=True)

In [42]:
g_model['splats'].keys()

odict_keys(['means', 'opacities', 'quats', 'scales', 'sh0', 'shN'])

In [43]:
N = g_model['splats']['shN'].shape[1]

In [44]:
shN_cols = np.array([[f'sh{i}_x', f'sh{i}_y', f'sh{i}_z'] for i in range(1, N + 1)]).flatten()

In [45]:
shN = g_model['splats']['shN'].reshape(len(g_model['splats']['means']), -1)

In [34]:
columns = np.array(['x', 'y', 'z', 'a', 'q0', 'q1', 'q2', 'q3', 's0', 's1', 's2', 'sh0_x', 'sh0_y', 'sh0_z'])
columns = np.concatenate((columns, shN_cols))

In [46]:
data = np.column_stack((g_model['splats']['means'].numpy(), 
                       g_model['splats']['opacities'].numpy(), 
                       g_model['splats']['quats'].numpy(), 
                       g_model['splats']['scales'].numpy(),
                       g_model['splats']['sh0'].reshape(-1, 3), 
                       shN))

In [47]:
data

array([[-0.04730337, -0.15343167,  0.38964266, ..., -0.00529645,
        -0.00493299, -0.00195461],
       [ 0.22355649, -0.1884563 ,  0.15883228, ...,  0.04605302,
         0.02724276,  0.02788799],
       [-0.10345538, -0.09819045,  0.31356713, ...,  0.00494115,
         0.00780313,  0.00343809],
       ...,
       [-0.12415215,  1.0988488 , -0.05502048, ..., -0.13743153,
        -0.12917116, -0.15007524],
       [ 0.51211417,  0.84834313, -0.29698887, ..., -0.03106887,
        -0.03240661, -0.03486897],
       [ 1.6731802 ,  1.6349938 ,  0.34147534, ..., -0.07000026,
        -0.06195371, -0.07859948]], dtype=float32)

In [48]:
columns

array(['x', 'y', 'z', 'a', 'q0', 'q1', 'q2', 'q3', 's0', 's1', 's2',
       'sh0_x', 'sh0_y', 'sh0_z', 'sh1_x', 'sh1_y', 'sh1_z', 'sh2_x',
       'sh2_y', 'sh2_z', 'sh3_x', 'sh3_y', 'sh3_z', 'sh4_x', 'sh4_y',
       'sh4_z', 'sh5_x', 'sh5_y', 'sh5_z', 'sh6_x', 'sh6_y', 'sh6_z',
       'sh7_x', 'sh7_y', 'sh7_z', 'sh8_x', 'sh8_y', 'sh8_z', 'sh9_x',
       'sh9_y', 'sh9_z', 'sh10_x', 'sh10_y', 'sh10_z', 'sh11_x', 'sh11_y',
       'sh11_z', 'sh12_x', 'sh12_y', 'sh12_z', 'sh13_x', 'sh13_y',
       'sh13_z', 'sh14_x', 'sh14_y', 'sh14_z', 'sh15_x', 'sh15_y',
       'sh15_z'], dtype='<U6')

In [50]:
pd_data = pd.DataFrame(data, columns=columns)
pd_data

,x,y,z,a,q0,q1,q2,q3,s0,s1,...,sh12_z,sh13_x,sh13_y,sh13_z,sh14_x,sh14_y,sh14_z,sh15_x,sh15_y,sh15_z
0,-0.047303,-0.153432,0.389643,0.580064,0.969980,1.064052,0.494045,0.711231,-3.685590,-4.500667,...,-0.023774,-0.014860,-0.010231,-0.011014,0.003031,0.003139,0.002241,-0.005296,-0.004933,-0.001955
1,0.223556,-0.188456,0.158832,0.442476,0.600435,0.549816,0.419183,0.635242,-6.250227,-5.259087,...,0.009574,0.041071,0.037175,0.039994,-0.007116,-0.000891,0.003032,0.046053,0.027243,0.027888
2,-0.103455,-0.098190,0.313567,-0.329361,1.022540,-0.340142,1.042773,0.248610,-4.581302,-5.602844,...,0.017962,-0.023454,-0.035076,-0.031963,0.008128,0.015435,0.012516,0.004941,0.007803,0.003438
3,-0.463576,0.040361,-0.044645,-0.559555,0.525208,0.881674,0.826888,0.228393,-6.516395,-4.642702,...,-0.030417,0.012859,0.013521,0.011886,0.029247,0.032881,0.031078,0.018153,-0.000230,0.002419
4,-0.309236,-0.047417,0.250886,-2.851249,0.749165,0.635734,0.080704,0.848570,-6.381673,-6.720093,...,-0.063669,-0.040614,-0.052224,-0.059923,-0.027812,-0.019891,-0.041487,-0.016731,-0.030924,-0.020903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530055,-1.231520,-1.595784,0.658034,-1.801160,0.371741,0.442785,0.172438,0.645947,-6.194151,-7.226207,...,0.005511,-0.030233,-0.016444,-0.034237,0.046166,0.028830,0.050376,0.007612,0.010594,0.018105
530056,2.033041,-0.039398,0.258939,-1.017034,0.683827,0.528958,0.312445,0.870445,-4.415745,-6.053058,...,-0.030856,0.032131,0.032617,0.030509,0.032901,0.032876,0.030922,-0.032247,-0.032686,-0.030671
530057,-0.124152,1.098849,-0.055020,6.630549,0.186276,0.315922,0.854295,0.610843,-9.276470,-4.038289,...,0.143839,-0.135638,-0.129997,-0.150324,0.118197,0.101019,0.121325,-0.137432,-0.129171,-0.150075
530058,0.512114,0.848343,-0.296989,4.012188,0.559892,0.166613,0.317968,0.794656,-4.299723,-6.916945,...,-0.022788,0.059990,0.073307,0.068385,-0.063135,-0.076341,-0.089985,-0.031069,-0.032407,-0.034869


In [52]:
cloud = pyntcloud.PyntCloud(pd_data)

In [53]:
cloud.points

,x,y,z,a,q0,q1,q2,q3,s0,s1,...,sh12_z,sh13_x,sh13_y,sh13_z,sh14_x,sh14_y,sh14_z,sh15_x,sh15_y,sh15_z
0,-0.047303,-0.153432,0.389643,0.580064,0.969980,1.064052,0.494045,0.711231,-3.685590,-4.500667,...,-0.023774,-0.014860,-0.010231,-0.011014,0.003031,0.003139,0.002241,-0.005296,-0.004933,-0.001955
1,0.223556,-0.188456,0.158832,0.442476,0.600435,0.549816,0.419183,0.635242,-6.250227,-5.259087,...,0.009574,0.041071,0.037175,0.039994,-0.007116,-0.000891,0.003032,0.046053,0.027243,0.027888
2,-0.103455,-0.098190,0.313567,-0.329361,1.022540,-0.340142,1.042773,0.248610,-4.581302,-5.602844,...,0.017962,-0.023454,-0.035076,-0.031963,0.008128,0.015435,0.012516,0.004941,0.007803,0.003438
3,-0.463576,0.040361,-0.044645,-0.559555,0.525208,0.881674,0.826888,0.228393,-6.516395,-4.642702,...,-0.030417,0.012859,0.013521,0.011886,0.029247,0.032881,0.031078,0.018153,-0.000230,0.002419
4,-0.309236,-0.047417,0.250886,-2.851249,0.749165,0.635734,0.080704,0.848570,-6.381673,-6.720093,...,-0.063669,-0.040614,-0.052224,-0.059923,-0.027812,-0.019891,-0.041487,-0.016731,-0.030924,-0.020903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530055,-1.231520,-1.595784,0.658034,-1.801160,0.371741,0.442785,0.172438,0.645947,-6.194151,-7.226207,...,0.005511,-0.030233,-0.016444,-0.034237,0.046166,0.028830,0.050376,0.007612,0.010594,0.018105
530056,2.033041,-0.039398,0.258939,-1.017034,0.683827,0.528958,0.312445,0.870445,-4.415745,-6.053058,...,-0.030856,0.032131,0.032617,0.030509,0.032901,0.032876,0.030922,-0.032247,-0.032686,-0.030671
530057,-0.124152,1.098849,-0.055020,6.630549,0.186276,0.315922,0.854295,0.610843,-9.276470,-4.038289,...,0.143839,-0.135638,-0.129997,-0.150324,0.118197,0.101019,0.121325,-0.137432,-0.129171,-0.150075
530058,0.512114,0.848343,-0.296989,4.012188,0.559892,0.166613,0.317968,0.794656,-4.299723,-6.916945,...,-0.022788,0.059990,0.073307,0.068385,-0.063135,-0.076341,-0.089985,-0.031069,-0.032407,-0.034869


In [54]:
cloud.to_file('./south-building.ply')

C:\Users\julia\anaconda3\envs\ptcl\lib\site-packages\pyntcloud\io\ply.py:260: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = property_formats[str(df.dtypes[i])[0]]
C:\Users\julia\anaconda3\envs\ptcl\lib\site-packages\pyntcloud\io\ply.py:260: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = property_formats[str(df.dtypes[i])[0]]
C:\Users\julia\anaconda3\envs\ptcl\lib\site-packages\pyntcloud\io\ply.py:260: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, 